In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
with open('train/left.txt') as fopen:
    left = fopen.read().split('\n')

In [3]:
import malaya

/home/husein/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [4]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from malaya.text.function import replace_punct
from collections import defaultdict
import re
import random
import json

In [5]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_

In [6]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 5, max_replace = 15):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        try:
            if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
                continue
            elif splitted_left[l].isupper() or splitted_right[r].isupper():
                continue
            elif splitted_left[l].lower() == splitted_right[r].lower():
                continue
            elif splitted_left[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
                continue
            else:
                selected_alignment.append((l, r))
        except:
            pass
    
    try:
        count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
        selected = random.sample(selected_alignment, count_replace)
        for s in selected:
            splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])

        return ' '.join(splitted_left), selected
    
    except:
        return ' '.join(splitted_left), []

In [7]:
model = malaya.language_detection.substring_rules()

In [ ]:
fast_text = malaya.language_detection.fasttext()

In [8]:
tokenizer = malaya.tokenizer.Tokenizer()

In [14]:
malaya.text.function.is_malay('terminal')

True

In [15]:
l = tokenizer.tokenize(left[0])
predicted = model.predict(l)
for no, p in enumerate(predicted):
    if p == 'EN' and malaya.text.function.is_malay(l[no].lower()):
        predicted[no] = 'MS'
predicted

['MS',
 'NOT_LANG',
 'CAPITAL',
 'MS',
 'MS',
 'NOT_LANG',
 'MS',
 'MS',
 'MS',
 'NOT_LANG',
 'NOT_LANG',
 'OTHERS',
 'EN',
 'MS',
 'MS',
 'MS',
 'NOT_LANG',
 'MS',
 'MS',
 'MS',
 'MS',
 'NOT_LANG']

In [18]:
list(zip(l, predicted))

[('Terminal', 'MS'),
 ('1', 'NOT_LANG'),
 ('KKIA', 'CAPITAL'),
 ('dilengkapi', 'MS'),
 ('kemudahan', 'MS'),
 ('64', 'NOT_LANG'),
 ('kaunter', 'MS'),
 ('daftar', 'MS'),
 ('masuk', 'MS'),
 (',', 'NOT_LANG'),
 ('12', 'NOT_LANG'),
 ('aero', 'OTHERS'),
 ('bridge', 'EN'),
 ('selain', 'MS'),
 ('mampu', 'MS'),
 ('menampung', 'MS'),
 ('3,200', 'NOT_LANG'),
 ('penumpang', 'MS'),
 ('dalam', 'MS'),
 ('satu', 'MS'),
 ('masa', 'MS'),
 ('.', 'NOT_LANG')]

In [37]:
from tqdm import tqdm

def loop(rows):
    rows, _ = rows
    strings, labels = [], []
    for i in tqdm(range(len(rows))):
        try:
            left_ = rows[i]
            tokenized_l = tokenizer.tokenize(left_)
            l = tokenized_l
            predicted = model.predict(l)
            for no, p in enumerate(predicted):
                if p == 'EN' and malaya.text.function.is_malay(l[no].lower()):
                    predicted[no] = 'MS'
                elif p == 'OTHERS' and fast_text.predict([l[no]])[0] in ['malay', 'ind']:
                    predicted[no] = 'MS'
            strings.append(l)
            labels.append(predicted)
        except Exception as e:
            print(e)
    
    return [[strings, labels]]

In [43]:
r = loop((left[399999:400000],0))

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2340.57it/s]


In [44]:
r[0][0][-1]

['Wahai',
 'Tuhanku',
 ',',
 'berhala-berhala',
 'itu',
 'telah',
 'menyebabkan',
 'sesat',
 'banyak',
 'di',
 'antara',
 'umat',
 'manusia',
 '.']

In [45]:
r[0][1][-1]

['MS',
 'MS',
 'NOT_LANG',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'NOT_LANG']

In [28]:
import mp

In [46]:
r = mp.multiprocessing(left[:400000], loop, cores = 4)

100%|█████████████████████████████████| 100000/100000 [00:22<00:00, 4483.88it/s]


In [47]:
strings, labels = [], []
for i in range(len(r)):
    print(i, len(r[i][0]))
    strings.extend(r[i][0])
    labels.extend(r[i][1])

0 100000
1 100000
2 100000
3 100000


In [48]:
strings[-1], labels[-1]

(['Wahai',
  'Tuhanku',
  ',',
  'berhala-berhala',
  'itu',
  'telah',
  'menyebabkan',
  'sesat',
  'banyak',
  'di',
  'antara',
  'umat',
  'manusia',
  '.'],
 ['MS',
  'MS',
  'NOT_LANG',
  'MS',
  'MS',
  'MS',
  'MS',
  'MS',
  'MS',
  'MS',
  'MS',
  'MS',
  'MS',
  'NOT_LANG'])

In [49]:
with open('ms-substrings.json', 'w') as fopen:
    json.dump({'strings': strings, 'labels': labels}, fopen)